# Regularization Demo
## Ridge, Lasso y Elastic Net
### 1. Cargamos librerías

### 2. Vemos qué pinta tiene el dataset

### 3. Rápido análisis exploratorio para ver sus dependencias lineales

### 4. Preprocesado de los datos

### 5. Regresión lineal múltiple

Pintamos los coeficientes para verlos mejor

### 6. Ridge regression

Veamos en la gráfica cómo quedaría para alpha = 1

Si hacemos ahora alpha = 10

### 7. Lasso regression

### 8. Elastic Net

Conclusion :
From the above analysis we can reach the following conclusion about different regularization methods:

Regularization is used to reduce the dependence on any particular independent variable by adding the penalty term to the Loss function. This term prevents the coefficients of the independent variables to take extreme values.
Ridge Regression adds L2 regularization penalty term to loss function. This term reduces the coefficients but does not make them 0 and thus doesn’t eliminate any independent variable completely. It can be used to measure the impact of the different independent variables.
Lasso Regression adds L1 regularization penalty term to loss function. This term reduces the coefficients as well as makes them 0 thus effectively eliminate the corresponding independent variable completely. It can be used for feature selection etc.
Elastic Net is a combination of both of the above regularization. It contains both the L1 and L2 as its penalty term. It performs better than Ridge and Lasso Regression for most of the test cases.
